[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/introduction/18_Deduplication.ipynb)

# Deduplication Module

## What is the Deduplication Module?

The Deduplication Module is a comprehensive system for identifying and merging duplicate entities in knowledge graphs. It helps maintain data quality by detecting semantically similar entities, calculating similarity scores, and merging duplicates into canonical representations while preserving provenance and handling conflicts.

**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/deduplication/)

## Module Capabilities

### Core Functionality

1. **Similarity Calculation**
   - Multiple algorithms: Exact matching, Levenshtein distance, Jaro-Winkler, Cosine similarity, Jaccard similarity
   - Multi-factor aggregation: Combines string, property, relationship, and embedding similarity
   - Configurable weights for different similarity components
   - Batch processing for efficient pairwise comparisons

2. **Duplicate Detection**
   - Pairwise comparison: Compare all entity pairs for duplicates
   - Group detection: Find clusters of duplicates using Union-Find algorithm
   - Incremental detection: Efficiently detect duplicates between new and existing entities
   - Confidence scoring: Multi-factor confidence calculation (similarity + name match + property matches)
   - Relationship duplicate detection: Identify duplicate relationships

3. **Entity Merging**
   - Multiple merge strategies: Keep first, last, most complete, highest confidence, or merge all
   - Automatic duplicate detection before merging
   - Conflict resolution: Handle property and relationship conflicts
   - Provenance preservation: Track which entities were merged
   - Merge history: Maintain record of all merge operations
   - Quality validation: Validate merged entities for completeness

4. **Clustering**
   - Graph-based clustering: Union-Find algorithm for connected components
   - Hierarchical clustering: Agglomerative clustering for large datasets
   - Cluster quality metrics: Cohesion and separation measures
   - Incremental updates: Update clusters with new entities

5. **Advanced Features**
   - Property-specific merge rules: Different strategies for different properties
   - Custom conflict resolution: Define custom functions for resolving conflicts
   - Method registry: Register and use custom deduplication methods
   - Configuration management: Centralized configuration from multiple sources
   - Extensibility: Add custom similarity, detection, merge, and clustering methods

## Module Architecture

### Main Components

**Core Classes:**
- `DuplicateDetector`: Detects duplicate entities and relationships
- `EntityMerger`: Merges duplicate entities with configurable strategies
- `SimilarityCalculator`: Calculates multi-factor similarity between entities
- `ClusterBuilder`: Builds clusters for efficient batch deduplication
- `MergeStrategyManager`: Manages merge strategies and conflict resolution
- `MethodRegistry`: Registry for custom deduplication methods
- `DeduplicationConfig`: Centralized configuration management

**Data Structures:**
- `DuplicateCandidate`: Duplicate pair with confidence scores
- `DuplicateGroup`: Group of duplicate entities
- `MergeOperation`: Merge operation record with metadata
- `SimilarityResult`: Similarity calculation result with components
- `Cluster`: Entity cluster representation
- `ClusterResult`: Cluster building result with quality metrics
- `MergeResult`: Merge operation result with conflicts
- `MergeStrategy`: Enumeration of merge strategies

**Convenience Functions:**
- `detect_duplicates()`: Duplicate detection wrapper
- `merge_entities()`: Entity merging wrapper
- `calculate_similarity()`: Similarity calculation wrapper
- `build_clusters()`: Cluster building wrapper
- `get_deduplication_method()`: Get method by name
- `list_available_methods()`: List all available methods

## Algorithms Used

**Similarity Calculation:**
- Levenshtein Distance: Dynamic programming for edit distance
- Jaro Similarity: Character-based similarity with match window
- Jaro-Winkler: Jaro with prefix bonus (up to 4 characters)
- Cosine Similarity: Vector dot product for embeddings
- Jaccard Similarity: Intersection over union for sets
- Multi-factor Aggregation: Weighted sum of similarity components

**Duplicate Detection:**
- Pairwise Comparison: O(n²) all-pairs similarity calculation
- Union-Find Algorithm: Disjoint set union for group formation
- Confidence Scoring: Multi-factor confidence calculation
- Incremental Processing: O(n×m) efficient new vs existing comparison

**Clustering:**
- Union-Find (DSU): Connected component detection
- Hierarchical Clustering: Agglomerative bottom-up clustering
- Similarity Graph: Graph construction from similarity scores

**Entity Merging:**
- Strategy Pattern: Multiple merge strategies
- Conflict Resolution: Voting, credibility-weighted, temporal, confidence-based
- Property Merging: Rule-based property combination
- Provenance Tracking: Metadata preservation during merges

## Installation

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```

## Table of Contents

1. [Module Overview](#module-overview)
2. [Setup and Sample Data](#setup)
3. [Similarity Calculation](#similarity)
4. [Duplicate Detection](#detection)
5. [Entity Merging](#merging)
6. [Clustering](#clustering)
7. [Advanced Features](#advanced)
8. [Complete Workflow](#workflow)

---

## Module Overview {#module-overview}

The deduplication module provides a complete solution for maintaining clean knowledge graphs by identifying and merging duplicate entities. It supports multiple similarity algorithms, detection methods, merge strategies, and clustering approaches, making it suitable for various use cases from simple exact matching to advanced semantic deduplication.


In [ ]:
%pip install -U "semantica[all]"\nimport semantica\nprint(semantica.__version__)\n

In [ ]:
# Import all deduplication classes
from semantica.deduplication import (
    # Main Classes
    DuplicateDetector,
    EntityMerger,
    SimilarityCalculator,
    ClusterBuilder,
    MergeStrategyManager,
    MethodRegistry,
    DeduplicationConfig,
    # Data Classes
    DuplicateCandidate,
    DuplicateGroup,
    MergeOperation,
    SimilarityResult,
    Cluster,
    ClusterResult,
    MergeResult,
    MergeStrategy,
    # Global Instances
    method_registry,
    dedup_config,
)

# Create sample entities with potential duplicates
entities = [
    {
        "id": "e1",
        "name": "Apple Inc.",
        "type": "Company",
        "founded": 1976,
        "properties": {"industry": "Technology", "headquarters": "Cupertino"},
        "relationships": [{"subject": "e1", "predicate": "founded_by", "object": "Steve Jobs"}],
    },
    {
        "id": "e2",
        "name": "Apple Inc",
        "type": "Company",
        "founded": 1976,
        "properties": {"industry": "Tech", "headquarters": "Cupertino, CA"},
        "relationships": [{"subject": "e2", "predicate": "founded_by", "object": "Steve Jobs"}],
    },
    {
        "id": "e3",
        "name": "Microsoft Corporation",
        "type": "Company",
        "founded": 1975,
        "properties": {"industry": "Technology", "headquarters": "Redmond"},
    },
    {
        "id": "e4",
        "name": "Microsoft",
        "type": "Company",
        "founded": 1975,
        "properties": {"industry": "Tech", "headquarters": "Redmond, WA"},
    },
    {
        "id": "e5",
        "name": "Google LLC",
        "type": "Company",
        "founded": 1998,
        "properties": {"industry": "Technology"},
    },
]

print(f"Created {len(entities)} sample entities")
print("\nEntity names:")
for e in entities:
    print(f"  - {e['name']} (ID: {e['id']})")


---

## Similarity Calculation {#similarity}

The module provides multiple algorithms for calculating similarity between entities. Similarity is the foundation of duplicate detection.

### Available Similarity Methods

1. **Exact Matching**: Binary match/no-match for identical strings
2. **Levenshtein Distance**: Edit distance between strings (insertions, deletions, substitutions)
3. **Jaro-Winkler**: Character-based similarity with prefix bonus for common prefixes
4. **Cosine Similarity**: Vector similarity for embeddings using dot product
5. **Jaccard Similarity**: Set-based similarity (intersection over union)
6. **Property Similarity**: Weighted comparison of property values
7. **Relationship Similarity**: Jaccard similarity of relationship sets
8. **Multi-factor Similarity**: Weighted aggregation of all components

### SimilarityCalculator Class

The `SimilarityCalculator` class provides a unified interface for all similarity calculations with configurable weights for different components.


In [ ]:
### Example: Duplicate Detection

# Initialize DuplicateDetector
detector = DuplicateDetector(
    similarity_threshold=0.7,
    confidence_threshold=0.6,
    use_clustering=True,
)

# Detect duplicate candidates (pairwise)
candidates = detector.detect_duplicates(entities)
print(f"Found {len(candidates)} duplicate candidate(s)")
for candidate in candidates:
    print(f"  {candidate.entity1['name']} <-> {candidate.entity2['name']}")
    print(f"    Similarity: {candidate.similarity_score:.3f}, Confidence: {candidate.confidence:.3f}")

# Detect duplicate groups
duplicate_groups = detector.detect_duplicate_groups(entities)
print(f"\nFound {len(duplicate_groups)} duplicate group(s)")
for i, group in enumerate(duplicate_groups, 1):
    print(f"  Group {i}: {[e['name'] for e in group.entities]} (confidence: {group.confidence:.3f})")

# Incremental detection
existing_entities = entities[:3]
new_entities = entities[3:]
incremental_candidates = detector.incremental_detect(new_entities, existing_entities, threshold=0.7)
print(f"\nFound {len(incremental_candidates)} incremental duplicate(s)")
for candidate in incremental_candidates:
    print(f"  {candidate.entity1['name']} duplicates {candidate.entity2['name']} (confidence: {candidate.confidence:.3f})")


---

## Duplicate Detection {#detection}

The module detects duplicate entities using similarity metrics and confidence scoring. Detection can be performed pairwise, in groups, or incrementally.

### Detection Methods

1. **Pairwise Detection**: Compare all entity pairs (O(n²) complexity)
2. **Group Detection**: Find clusters of duplicates using Union-Find algorithm
3. **Incremental Detection**: Efficiently detect duplicates between new and existing entities (O(n×m))
4. **Relationship Detection**: Identify duplicate relationships

### Confidence Scoring

The module calculates confidence scores using multiple factors:
- Similarity score between entities
- Name matching (exact or fuzzy)
- Property value matches
- Entity type matches
- Relationship overlap

### DuplicateDetector Class

The `DuplicateDetector` class provides all duplicate detection capabilities with configurable thresholds and clustering options.


In [ ]:
### Example: Similarity Calculation

# Initialize SimilarityCalculator
calculator = SimilarityCalculator(
    string_weight=0.4,
    property_weight=0.3,
    relationship_weight=0.2,
    embedding_weight=0.1,
)

# Calculate overall similarity (multi-factor)
entity1, entity2 = entities[0], entities[1]
result = calculator.calculate_similarity(entity1, entity2)
print(f"Overall Similarity: {result.score:.3f}")
print(f"Components: {result.components}")

# String similarity methods
str1, str2 = "Apple Inc.", "Apple Inc"
for method in ["levenshtein", "jaro_winkler", "cosine"]:
    score = calculator.calculate_string_similarity(str1, str2, method=method)
    print(f"{method}: {score:.3f}")

# Property and relationship similarity
prop_score = calculator.calculate_property_similarity(entity1, entity2)
rel_score = calculator.calculate_relationship_similarity(entity1, entity2)
print(f"\nProperty Similarity: {prop_score:.3f}")
print(f"Relationship Similarity: {rel_score:.3f}")

# Batch similarity calculation
similarity_pairs = calculator.batch_calculate_similarity(entities, threshold=0.5)
print(f"\nFound {len(similarity_pairs)} similar pairs (threshold >= 0.5)")
for e1, e2, score in similarity_pairs:
    print(f"  {e1['name']} <-> {e2['name']}: {score:.3f}")


---

## Entity Merging {#merging}

The module merges duplicate entities into single canonical representations using configurable strategies. Merging preserves provenance, handles conflicts, and maintains merge history.

### Merge Strategies

1. **KEEP_FIRST**: Preserve the first entity encountered, merge others into it
2. **KEEP_LAST**: Preserve the last entity encountered, merge others into it
3. **KEEP_MOST_COMPLETE**: Preserve entity with most properties and relationships
4. **KEEP_HIGHEST_CONFIDENCE**: Preserve entity with highest confidence score
5. **MERGE_ALL**: Create new entity combining all properties and relationships
6. **CUSTOM**: User-defined merge logic

### Conflict Resolution

When merging entities with conflicting property values, the module supports:
- Voting: Majority value selection
- Credibility-weighted: Weighted by source credibility
- Temporal: Most recent value
- Confidence-based: Highest confidence value
- Custom functions: User-defined resolution logic

### EntityMerger Class

The `EntityMerger` class provides entity merging with automatic duplicate detection, provenance preservation, and merge history tracking.

### MergeStrategyManager Class

The `MergeStrategyManager` class provides advanced merge management with property-specific rules and custom conflict resolution functions.


In [ ]:
### Example: Entity Merging

# Initialize EntityMerger
merger = EntityMerger(preserve_provenance=True)

# Merge duplicates (automatic detection)
merge_operations = merger.merge_duplicates(entities)
print(f"Original entities: {len(entities)}")
print(f"Merge operations: {len(merge_operations)}")
for i, op in enumerate(merge_operations, 1):
    print(f"  Operation {i}: Merged {len(op.source_entities)} entities → {op.merged_entity.get('name')}")
    if op.merge_result.conflicts:
        print(f"    Conflicts: {len(op.merge_result.conflicts)}")

# Merge with specific strategy
operations = merger.merge_duplicates(entities, strategy=MergeStrategy.KEEP_MOST_COMPLETE)
print(f"\nMerged using KEEP_MOST_COMPLETE: {len(operations)} operations")

# Merge specific group
duplicate_entities = [entities[0], entities[1]]
operation = merger.merge_entity_group(duplicate_entities, strategy=MergeStrategy.KEEP_FIRST)
print(f"\nMerged group: {[e['name'] for e in operation.source_entities]} → {operation.merged_entity['name']}")

# Get merge history
history = merger.get_merge_history()
print(f"\nTotal merge operations in history: {len(history)}")

# Validate merge quality
if operations:
    validation = merger.validate_merge_quality(operations[0])
    print(f"\nValidation: Valid={validation['valid']}, Quality={validation['quality_score']:.3f}")


---

## Clustering {#clustering}

The module provides clustering capabilities for efficient batch deduplication of large datasets. Clustering groups similar entities together before deduplication.

### Clustering Methods

1. **Graph-Based Clustering**: Union-Find algorithm for connected components
   - Builds similarity graph from pairwise similarities
   - Uses Union-Find (Disjoint Set Union) for efficient component detection
   - Suitable for medium-sized datasets

2. **Hierarchical Clustering**: Agglomerative bottom-up clustering
   - Builds cluster hierarchy by merging similar clusters
   - Suitable for large datasets
   - Provides cluster quality metrics (cohesion, separation)

### Cluster Quality Metrics

- **Cohesion**: Average similarity within clusters
- **Separation**: Average similarity between clusters
- **Cluster Quality Score**: Combined metric for cluster evaluation

### ClusterBuilder Class

The `ClusterBuilder` class provides clustering with configurable thresholds, quality metrics, and incremental update capabilities.


In [ ]:
### Example: Advanced Merge Strategies

# Initialize MergeStrategyManager
strategy_manager = MergeStrategyManager(default_strategy="keep_most_complete")

# Add property-specific rules
strategy_manager.add_property_rule("name", MergeStrategy.KEEP_FIRST, priority=1)
strategy_manager.add_property_rule("description", MergeStrategy.MERGE_ALL, priority=1)

# Custom conflict resolution
def resolve_longest(values):
    return max(values, key=len)

strategy_manager.add_property_rule(
    "headquarters", MergeStrategy.CUSTOM, conflict_resolution=resolve_longest, priority=2
)

print("Added merge rules: name=KEEP_FIRST, description=MERGE_ALL, headquarters=CUSTOM")

# Merge entities with property rules
duplicate_pair = [entities[0], entities[1]]
merge_result = strategy_manager.merge_entities(duplicate_pair)
print(f"\nMerged Entity: {merge_result.merged_entity.get('name')}")
print(f"Conflicts: {len(merge_result.conflicts)}")

# Validate merge quality
validation = strategy_manager.validate_merge(merge_result)
print(f"Validation: Valid={validation['valid']}, Quality={validation['quality_score']:.3f}")


---

## Advanced Features {#advanced}

The module provides advanced features for extensibility, configuration, and custom method registration.

### Configuration Management

The `DeduplicationConfig` class provides centralized configuration from multiple sources:
- Programmatic configuration (via `set()` method)
- Environment variables
- Configuration files (YAML, JSON, TOML)
- Default values

Configuration can be set globally or per-method for fine-grained control.

### Method Registry

The `MethodRegistry` class allows registration of custom deduplication methods:
- Custom similarity calculation methods
- Custom duplicate detection methods
- Custom entity merging methods
- Custom clustering methods

Registered methods can be used throughout the module via the registry.

### Property-Specific Merge Rules

The `MergeStrategyManager` supports property-specific merge rules:
- Different merge strategies for different properties
- Custom conflict resolution functions per property
- Priority-based rule application
- Flexible rule composition


In [ ]:
### Example: Clustering

# Initialize ClusterBuilder
cluster_builder = ClusterBuilder(
    similarity_threshold=0.7,
    min_cluster_size=2,
    max_cluster_size=100,
    use_hierarchical=False,
)

# Graph-based clustering
cluster_result = cluster_builder.build_clusters(entities)
print(f"Clusters: {len(cluster_result.clusters)}, Unclustered: {len(cluster_result.unclustered)}")
print(f"Quality Metrics: {cluster_result.quality_metrics}")
for cluster in cluster_result.clusters:
    print(f"  {cluster.cluster_id}: {len(cluster.entities)} entities (quality: {cluster.quality_score:.3f})")

# Hierarchical clustering
hierarchical_builder = ClusterBuilder(similarity_threshold=0.7, use_hierarchical=True)
hierarchical_result = hierarchical_builder.build_clusters(entities)
print(f"\nHierarchical Clustering: {len(hierarchical_result.clusters)} clusters")
print(f"Quality Metrics: {hierarchical_result.quality_metrics}")


---

## Complete Workflow {#workflow}

A typical deduplication workflow involves:
1. Configuration: Set similarity thresholds and method preferences
2. Clustering: Build clusters of similar entities (optional, for large datasets)
3. Detection: Identify duplicate entities within clusters or entire dataset
4. Merging: Merge duplicate entities using appropriate strategies
5. Validation: Validate merge quality and completeness
6. History: Track merge operations for audit and rollback

The module provides both class-based and function-based interfaces for flexibility.


In [ ]:
### Example: Configuration and Method Registry

# Configuration management
threshold = dedup_config.get("similarity_threshold", default=0.7)
confidence = dedup_config.get("confidence_threshold", default=0.6)
print(f"Current: similarity_threshold={threshold}, confidence_threshold={confidence}")

# Set configuration programmatically
dedup_config.set("similarity_threshold", 0.8)
dedup_config.set("confidence_threshold", 0.7)
print(f"Updated: similarity_threshold={dedup_config.get('similarity_threshold')}")

# Method-specific configuration
dedup_config.set_method_config("levenshtein", case_sensitive=False)
levenshtein_config = dedup_config.get_method_config("levenshtein")
print(f"Method config (levenshtein): {levenshtein_config}")

# Custom method registration
def word_overlap_similarity(entity1, entity2, **kwargs):
    """Custom similarity based on word overlap."""
    name1 = entity1.get("name", "").lower().split()
    name2 = entity2.get("name", "").lower().split()
    
    if not name1 or not name2:
        return SimilarityResult(score=0.0, method="word_overlap")
    
    set1, set2 = set(name1), set(name2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    score = intersection / union if union > 0 else 0.0
    return SimilarityResult(score=score, method="word_overlap")

# Register custom method
method_registry.register("similarity", "word_overlap", word_overlap_similarity)
print("\nRegistered 'word_overlap' similarity method")

# Use custom method
custom_method = method_registry.get("similarity", "word_overlap")
if custom_method:
    result = custom_method(entities[0], entities[1])
    print(f"Word Overlap Similarity: {result.score:.3f}")

# List all registered methods
all_registered = method_registry.list_all()
print(f"\nRegistered Methods: {all_registered}")


---

## Practical Examples

The following sections demonstrate practical usage of the module components.


In [ ]:
### Example: Complete Workflow

# Complete deduplication workflow

# Step 1: Configure
dedup_config.set("similarity_threshold", 0.75)
dedup_config.set("confidence_threshold", 0.65)

# Step 2: Build clusters
cluster_builder = ClusterBuilder(similarity_threshold=0.75, min_cluster_size=2, max_cluster_size=50)
cluster_result = cluster_builder.build_clusters(entities)
print(f"Step 1: Created {len(cluster_result.clusters)} clusters")

# Step 3: Detect duplicates
detector = DuplicateDetector(similarity_threshold=0.75, confidence_threshold=0.65)
all_duplicate_groups = []
for cluster in cluster_result.clusters:
    groups = detector.detect_duplicate_groups(cluster.entities)
    all_duplicate_groups.extend(groups)
print(f"Step 2: Found {len(all_duplicate_groups)} duplicate groups")

# Step 4: Merge duplicates
merger = EntityMerger(preserve_provenance=True)
merge_operations = merger.merge_duplicates(entities, strategy=MergeStrategy.KEEP_MOST_COMPLETE)
print(f"Step 3: Performed {len(merge_operations)} merge operations")

# Step 5: Extract results
merged_entities = [op.merged_entity for op in merge_operations]
print(f"\nResults:")
print(f"  Original: {len(entities)} entities")
print(f"  Merged: {len(merged_entities)} entities")
print(f"  Reduction: {len(entities) - len(merged_entities)} entities")

# Step 6: Validate merge quality
for i, op in enumerate(merge_operations, 1):
    validation = merger.validate_merge_quality(op)
    print(f"  Merge {i}: Valid={validation['valid']}, Quality={validation['quality_score']:.3f}")


### Example: Similarity Calculation


In [ ]:
# Complete deduplication workflow

# Step 1: Configure
dedup_config.set("similarity_threshold", 0.75)
dedup_config.set("confidence_threshold", 0.65)

# Step 2: Build clusters
cluster_builder = ClusterBuilder(similarity_threshold=0.75, min_cluster_size=2, max_cluster_size=50)
cluster_result = cluster_builder.build_clusters(entities)
print(f"Step 1: Created {len(cluster_result.clusters)} clusters")

# Step 3: Detect duplicates
detector = DuplicateDetector(similarity_threshold=0.75, confidence_threshold=0.65)
all_duplicate_groups = []
for cluster in cluster_result.clusters:
    groups = detector.detect_duplicate_groups(cluster.entities)
    all_duplicate_groups.extend(groups)
print(f"Step 2: Found {len(all_duplicate_groups)} duplicate groups")

# Step 4: Merge duplicates
merger = EntityMerger(preserve_provenance=True)
merge_operations = merger.merge_duplicates(entities, strategy=MergeStrategy.KEEP_MOST_COMPLETE)
print(f"Step 3: Performed {len(merge_operations)} merge operations")

# Step 5: Extract results
merged_entities = [op.merged_entity for op in merge_operations]
print(f"\nResults:")
print(f"  Original: {len(entities)} entities")
print(f"  Merged: {len(merged_entities)} entities")
print(f"  Reduction: {len(entities) - len(merged_entities)} entities")

# Step 6: Validate merge quality
for i, op in enumerate(merge_operations, 1):
    validation = merger.validate_merge_quality(op)
    print(f"  Merge {i}: Valid={validation['valid']}, Quality={validation['quality_score']:.3f}")


### Example: Duplicate Detection
